Importowanie potrzebnych bibliotek i wczytywanie danych

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/DawidMoza/psi/main/New_cars_price.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15,16,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Pierwsza kolumna zawiera nazwy modeli, usuńmy ją i zamiast niej używajmy indeksów

Przeanalizujmy dostępne kolumny i wybierzmy te które mają jakiekolwiek powiązanie z typem nadwozia, kolumną docelową będzie oczywiście "Body Style"

Pozostawny wyłącznie te, które jakkolwiek odnoszą się do konstrukcji nadwozia i podwozia, są to:

- Drivetrain
- Passenger Capacity
- Passenger Doors
- Base Curb Weight (lbs)
- Passenger Volume (ft³)
- Wheelbase (in)
- Track Width, Front (in)
- Height, Overall (in)
- Turning Diameter - Curb to Curb (ft)
- Front Tire Size

Uprośćmy również ich nazwy i usuńmy wiersze z brakującymi danymi.

In [ ]:
df.insert(0, "", range(len(df.index)), True)
df.set_index("", inplace=True)

df = df[["Drivetrain", "Passenger Capacity", "Passenger Doors", "Base Curb Weight (lbs)",
         "Passenger Volume (ft³)", "Wheelbase (in)", "Track Width, Front (in)", "Height, Overall (in)",
          "Turning Diameter - Curb to Curb (ft)", "Front Tire Size", "Body Style"]]
df.columns = ['Drivetrain', 'Passenger Capacity', 'Passenger Doors', 'Weight', 'Passenger Volume',
              'Wheelbase', 'Track Width', 'Height', 'Turning Diameter', 'Front Tire Size', 'Body Style']

df.dropna(inplace=True)

df.head()

,Drivetrain,Passenger Capacity,Passenger Doors,Weight,Passenger Volume,Wheelbase,Track Width,Height,Turning Diameter,Front Tire Size,Body Style
,,,,,,,,,,,
0,Front Wheel Drive,5,4,3790,104.0,108.3,64.2,65.7,39,P235/55HR19,Sport Utility
1,Front Wheel Drive,5,4,3829,104.0,108.3,64.2,65.7,39,P235/55HR19,Sport Utility
2,Front Wheel Drive,5,4,3821,104.0,108.3,64.2,65.7,39,P255/45VR20,Sport Utility
3,Front Wheel Drive,5,4,3783,104.0,108.3,64.2,65.7,39,P235/55HR19,Sport Utility
4,All Wheel Drive,5,4,4026,104.0,108.3,64.2,65.7,39,P235/55HR19,Sport Utility


Kolumna Front Tire Size zawiera dane, które nie są w formacie liczbowym, musimy je przekształcić. Najpierw sprawdźmy, czy wszystkie dane w tej kolumnie są prawidłowe, a następnie podzielmy ją na 3 inne kolumny, opisujące szerokość opony, jej profil, a także wielkość felgi.

Niektóre komórki zawierają dodatkowe oznaczenia, które należy w pierwszej kolejności usunąć.

In [ ]:
df.loc[~df["Front Tire Size"].str[0].str.isdigit(), "Front Tire Size"] = df.loc[~df["Front Tire Size"].str[0].str.isdigit(), "Front Tire Size"].str[1:]
df.loc[df["Front Tire Size"].str.len() == 10, "Front Tire Size"] = df.loc[df["Front Tire Size"].str.len() == 10, "Front Tire Size"].str[:6] + df.loc[df["Front Tire Size"].str.len() == 10, "Front Tire Size"].str[7:]
df.drop(df[df["Front Tire Size"].str.len() != 9].index, inplace=True)

tiresWidth = df["Front Tire Size"].str[:3]
tiresProfile = df["Front Tire Size"].str[4:6]
rimSize = df["Front Tire Size"].str[7:9]

df.insert(10, "Tire Width", tiresWidth, True)
df.insert(11, "Tire Profle", tiresProfile, True)
df.insert(12, "Rim Size", rimSize, True)

df.drop("Front Tire Size", axis=1, inplace=True)

df.head()

,Drivetrain,Passenger Capacity,Passenger Doors,Weight,Passenger Volume,Wheelbase,Track Width,Height,Turning Diameter,Tire Width,Tire Profle,Rim Size,Body Style
,,,,,,,,,,,,,
0,Front Wheel Drive,5,4,3790,104.0,108.3,64.2,65.7,39,235,55,19,Sport Utility
1,Front Wheel Drive,5,4,3829,104.0,108.3,64.2,65.7,39,235,55,19,Sport Utility
2,Front Wheel Drive,5,4,3821,104.0,108.3,64.2,65.7,39,255,45,20,Sport Utility
3,Front Wheel Drive,5,4,3783,104.0,108.3,64.2,65.7,39,235,55,19,Sport Utility
4,All Wheel Drive,5,4,4026,104.0,108.3,64.2,65.7,39,235,55,19,Sport Utility


Przeanalizujmy czy wartości pozostałe w kolumnach są na pewno prawidłowe, w tym celu wypiszmy wszytskie wartości unikalne w poszukiwaniu dziwnych, odstających danych.
  
Drivetrain

  - rodzaje napędu zapisane są w różne sposoby
  - '2-Wheel Drive' oraz '2 Wheel Drive' nie wskazują które koła są napędzane, zbyt ogólne dane

In [ ]:
print(df["Drivetrain"].unique())

df["Drivetrain"].replace("Front Wheel Drive", "0", inplace=True)
df["Drivetrain"].replace("All Wheel Drive", "3", inplace=True)
df["Drivetrain"].replace("4 Wheel Drive", "2", inplace=True)
df["Drivetrain"].replace("Rear Wheel Drive", "1", inplace=True)
df["Drivetrain"].replace("All-Wheel Drive", "3", inplace=True)
df["Drivetrain"].replace("Front-Wheel Drive", "0", inplace=True)
df["Drivetrain"].replace("All wheel drive", "3", inplace=True)
df["Drivetrain"].replace("All-wheel drive", "3", inplace=True)
df["Drivetrain"].replace("AWD", "3", inplace=True)
df["Drivetrain"].replace("4-Wheel Drive", "2", inplace=True)
df["Drivetrain"].replace("Rear wheel drive", "1", inplace=True)
df["Drivetrain"].replace("Front wheel drive", "0", inplace=True)
df["Drivetrain"].replace("Four Wheel Drive", "2", inplace=True)
df["Drivetrain"].replace("Front-wheel drive", "0", inplace=True)
df["Drivetrain"].replace("Four wheel drive", "2", inplace=True)
#df["Drivetrain"].replace("2-Wheel Drive", "0", inplace=True)
#df["Drivetrain"].replace("2 Wheel Drive", "0", inplace=True)
df["Drivetrain"].replace("Four-Wheel Drive", "2", inplace=True)
df["Drivetrain"].replace("Rear-Wheel Drive", "1", inplace=True)

df.drop(df[df["Drivetrain"] == "2-Wheel Drive"].index, inplace=True)
df.drop(df[df["Drivetrain"] == "2 Wheel Drive"].index, inplace=True)

df['Drivetrain'] = pd.to_numeric(df['Drivetrain'])

print(df["Drivetrain"].unique())

['Front Wheel Drive' 'All Wheel Drive' '4 Wheel Drive' 'Rear Wheel Drive'
 'All-Wheel Drive' 'Front-Wheel Drive' 'All wheel drive' 'All-wheel drive'
 'AWD' '4-Wheel Drive' 'Rear wheel drive' 'Front wheel drive'
 'Four Wheel Drive' 'Front-wheel drive' 'Four wheel drive' '2-Wheel Drive'
 '2 Wheel Drive' 'Four-Wheel Drive' 'Rear-Wheel Drive']
[0 3 2 1]


- samochody mogące pomieścić tylko 1 pasażera

In [ ]:
print(df["Passenger Capacity"].unique())

print(df.loc[df["Passenger Capacity"] == 1])

#przeszukanie danych zasugeruje nam, że jest to 2018 Dodge Challenger Specs: SRT Demon RWD, który w rzecywistości pomieści więcej pasażerów

df.drop(7061, inplace=True)

print(df["Passenger Capacity"].unique())

[5 7 6 2 4 8 1]
      Drivetrain  Passenger Capacity  Passenger Doors Weight  \
                                                               
7061           1                   1                2   4280   

      Passenger Volume  Wheelbase  Track Width Height Turning Diameter  \
                                                                         
7061              93.9      116.2         65.8   57.4               38   

     Tire Width Tire Profle Rim Size Body Style  
                                                 
7061        315          40       18    2dr Car  
[5 7 6 2 4 8]


- samochody z 3 drzwiami dla pasażerów

- zły format danych wagi

- zły format danych wysokości

- promienie zawracania zawierające jednostki, zły format danych

- szerokości opon z dodatkowymi oznaczeniami, zły format danych

- profile opon z dodatkowym oznaczeniem, zły format danych

- typy nadwozia niewskazujące na jego kształt